# 02 - Prétraitement et Feature Engineering

Ce notebook couvre l'étape de nettoyage et de création des features :
- Parsing du nombre d'avis
- Encodage du niveau du vendeur
- TF-IDF sur le titre
- Assemblage des features et extraction des cibles


## Import des librairies

In [ ]:
import os
import numpy as np
import pandas as pd
from scipy import sparse
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder

## Chargement du dataset Fiverr

In [ ]:
# Chemin vers le CSV nettoyé
data_path = os.path.join('data', 'fiverr_gigs', 'fiverr-data-gigs-cleaned.csv')
df = pd.read_csv(data_path, encoding='latin-1', low_memory=False)
print(df.head())

## Parsing du nombre d'avis

In [ ]:
def parse_reviews(x):
    if isinstance(x, str) and 'k' in x.lower():
        return int(x.lower().replace('k+', '000').replace('k', '000'))
    try:
        return int(x)
    except:
        return 0

df['num_reviews'] = df['Number of Reviewers'].apply(parse_reviews)
print(df[['Number of Reviewers', 'num_reviews']].head())

## Encodage one-hot du niveau du vendeur

In [ ]:
levels = df['Seller Level'].fillna('Inconnu').values.reshape(-1, 1)
ohe = OneHotEncoder(sparse=True, handle_unknown='ignore')
ohe.fit(levels)
X_level = ohe.transform(levels)
print("One-hot shape:", X_level.shape)

## TF-IDF sur le titre

In [ ]:
tfidf = TfidfVectorizer(max_features=2000, stop_words='english')
tfidf.fit(df['Title'].fillna(''))
X_title = tfidf.transform(df['Title'].fillna(''))
print("TF-IDF shape:", X_title.shape)

## Assemblage des features

In [ ]:
X_num = sparse.csr_matrix(df[['num_reviews']].values)
X = sparse.hstack([X_title, X_level, X_num], format='csr')
print("X total shape:", X.shape)

## Extraction des cibles

In [ ]:
y_price = df['Price (USD)'].values.astype(np.float32)
y_rating = df['Average Rating'].values.astype(np.float32)
label_map = {lvl: idx for idx, lvl in enumerate(ohe.categories_[0])}
y_level = df['Seller Level'].fillna('Inconnu').map(label_map).values
print(y_price[:5], y_rating[:5], y_level[:5])

## Sauvegarde des données préparées

In [ ]:
out_dir = os.path.join('data', 'fiverr_features')
os.makedirs(out_dir, exist_ok=True)
sparse.save_npz(os.path.join(out_dir, 'X_features.npz'), X)
np.save(os.path.join(out_dir, 'y_price.npy'), y_price)
np.save(os.path.join(out_dir, 'y_rating.npy'), y_rating)
np.save(os.path.join(out_dir, 'y_level.npy'), y_level)
print("Données sauvegardées dans", out_dir)